# Sistemas Inteligentes

## Exercício Computacional 2 - Classificação

Preencha todas as células adequadamente e responda todas as perguntas de forma completa e elaborada.

### Identificação do Aluno

#### Nome Completo

Daniel Escudero

#### RA

11005716

### Instruções

Treine e teste um classificador à sua escolha utilizando a base de dados **Olivetti Faces** (veja as referências).

Faça um caso com com *k-Fold* e um caso de forma direta, ou seja, utilizando o *train_test_split*.

Exiba a acurácia e a *Confusion Matrix*, então discuta sobre o desempenho do classificador para os dois casos.

Entre as células de códigos, insira células de tipo *Markdown* para fornecer explicações sobre os passos realizados; não insira apenas códigos "soltos" sem explicação.

Dica: desta vez, estamos lidando com uma quantidade relativamente grande de classes, então pode ser interessante exibir a *Confusion Matrix* utilizando escala de cinza para representar os valores de cada célula da matriz; para fazer isso, prefira utilizar a função **imshow** com o mapa de cores (argumento **cmap**) do tipo **'Greys'**.

**EXTRA**: tente fazer um caso com *Bootstrap* em vez de *k-Fold*, então comente sobre o método e os resultados.

## Desenvolvimento

In [51]:
# imports
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split, KFold

### Funções utilitarias:
- `getRawModel` retorna uma rede neural com parametros pre-definidos
- `extractDataFromIndexes` recebe uma lista de valores e uma lista de indices, e retorna uma lista com os valores nesses indices

In [66]:
# Funções utilitarias
def getRawModel():
    return MLPClassifier(hidden_layer_sizes=(300), warm_start=True)

def extractDataFromIndexes (data, indexes):
    return list(map(lambda i: data[i], indexes))

### Preparação de dados:
- Extração dos dados do sklearn;
- Reshaping das imagens;
- Divisao em dados de treino e dados de teste (25% para teste);

In [63]:
# Preparação de dados
rawData = fetch_olivetti_faces()

x = rawData.images
y = rawData.target
x = x.reshape(len(x), 64 * 64)

### Treino do classificador (forma direta):
- Separação dos dados em com `train_test_split`
- Criação do modelo (rede neural com parametros pre-definidos na funcao utilitaria `getRawModel`)
- Treino do modelo

In [64]:
# Treino do classificador

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
model = getRawModel()
model.fit(x_train, y_train)

/home/ufabc/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=300, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [67]:
model_kfold = getRawModel()
kfold = KFold(n_splits=5, shuffle=True)

# Divide dados de teste
for train, test in kfold.split(x):
    trainX = extractDataFromIndexes(x, train)
    testX = extractDataFromIndexes(x, test)
    trainY = extractDataFromIndexes(y, train)
    testY = extractDataFromIndexes(y, test)
    model_kfold.fit(trainX, trainY)

/home/ufabc/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Questões

### 1. O classificador escolhido por você obteve um bom desempenho em sua opinião? Por que?

ESCREVA AQUI A SUA RESPOSTA

### 2. Além da acurácia e da matriz de confusão, quais métricas e recursos poderiam ser utilizados para avaliar o seu classificador?

ESCREVA AQUI A SUA RESPOSTA

### 3. O seu classificador obteve um desempenho notoriamente superior ou inferior para alguma das classes? Para qual(is)? Por que você acredita que isso ocorreu?

ESCREVA AQUI A SUA RESPOSTA

### 4. O que se poderia fazer para melhorar o desempenho de seu classificador?

ESCREVA AQUI A SUA RESPOSTA

### 5. Se fosse utilizada apenas metade da base de dados deste exercício em vez de ela toda, mas mantendo as proporções para os dados de treinamento e teste, os resultados sofreriam alguma mudança significativa? Se sim, quais e em qual intensidade?

ESCREVA AQUI A SUA RESPOSTA

### 6. Para quê servem métodos como k-Fold e Bootstrap? Você observou alguma mudança significativa por conta de sua utilização? Se sim, qual? E a quê se deve isso?

ESCREVA AQUI A SUA RESPOSTA

## Referências

[Scikit-Learn - Datasets - Olivetti Faces](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_olivetti_faces.html)

[Matplotlib - Pyplot](https://matplotlib.org/api/pyplot_summary.html)

[Numpy](https://docs.scipy.org/doc/numpy/reference/)

[Scikit-Learn - Train, Test and Split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

[Scikit-Learn - KFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold)

[Scikit-Learn - Resample (Bootstrap)](http://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html)

[Scikit-Learn - KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

[Scikit-Learn - DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

[Scikit-Learn - LinearDiscriminantAnalysis](http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)

[Scikit-Learn - LinearSVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC)

[Scikit-Learn - MLPClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)

[Scikit-Learn - Accuracy Score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)

[Scikit-Learn - Confusion Matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)